In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.metrics import precision_recall_fscore_support

In [22]:
features = pd.read_csv("C:/Nini/Capstone/CSV_Files/ravdess_extracted_features_mel_mfcc_chroma.csv")
features = features.drop(features.columns[0], axis=1)
temp = shuffle(features)
df = np.random.rand(len(temp)) < 0.8
train = temp[df]
test = temp[~df]

In [23]:
X_train = np.array(train.iloc[:,:-1])
y_train = np.array(train.iloc[:,-1])
X_test = np.array(test.iloc[:,:-1])
y_test = np.array(test.iloc[:,-1])

In [24]:
lb = LabelEncoder()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

In [25]:
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.long)

In [26]:
batch_size = 256
train_loader = DataLoader(TensorDataset(X_train,y_train), batch_size=batch_size,shuffle=True)
test_loader = DataLoader(TensorDataset(X_test,y_test), batch_size=batch_size, shuffle=False)

In [27]:
class LSTMPyTorch(nn.Module):
    def __init__(self, input_size, hidden_sizes=[256, 256, 128], output_size=8, dropout=0.2, l2_reg=1e-5):
        super(LSTMPyTorch, self).__init__()
        
        self.lstm1 = nn.LSTM(input_size, hidden_sizes[0], batch_first=True)
        self.lstm2 = nn.LSTM(hidden_sizes[0], hidden_sizes[1], batch_first=True)
        self.lstm3 = nn.LSTM(hidden_sizes[1], hidden_sizes[2], batch_first=True)
        
        self.bn1 = nn.BatchNorm1d(hidden_sizes[0])
        self.bn2 = nn.BatchNorm1d(hidden_sizes[1])
        self.bn3 = nn.BatchNorm1d(hidden_sizes[2])

        self.fc1 = nn.Linear(hidden_sizes[2], 256)
        self.fc2 = nn.Linear(256, output_size)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.bn1(x.transpose(1,2)).transpose(1,2)

        x, _ = self.lstm2(x)
        x = self.bn2(x.transpose(1,2)).transpose(1,2)

        x, _ = self.lstm3(x)
        x = self.bn3(x.transpose(1,2)).transpose(1,2)
        
        x = self.relu(self.fc1(x[:, -1, :])) 
        x = self.dropout(x)
        x = self.softmax(self.fc2(x))
        return x

In [28]:
print("X_train shape:", X_train.shape)

X_train shape: torch.Size([1148, 1, 153])


In [29]:
input_size = 153
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMPyTorch(input_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

In [30]:
epochs = 1500
for epoch in range(epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    # Training loop
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)  # Get predicted class
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)
    
    train_accuracy = correct / total
    avg_train_loss = running_loss / len(train_loader)

    # Validation loop
    model.eval()
    val_correct = 0
    val_total = 0
    val_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for val_inputs, val_labels in test_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
            val_outputs = model(val_inputs)
            loss = criterion(val_outputs, val_labels)
            val_loss += loss.item()
            _, val_predicted = torch.max(val_outputs, 1)

            val_correct += (val_predicted == val_labels).sum().item()
            val_total += val_labels.size(0)

            # Store predictions and labels for evaluation
            all_preds.extend(val_predicted.cpu().numpy())
            all_labels.extend(val_labels.cpu().numpy())

    val_accuracy = val_correct / val_total
    avg_val_loss = val_loss / len(test_loader)

    # Compute Precision, Recall, F1-score
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted',zero_division=1)

    print(f"Epoch [{epoch+1}/{epochs}]")
    print(f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.4f}")
    print(f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.4f}")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")
    print("-" * 50)

Epoch [1/1500]
Train Loss: 2.0364, Train Acc: 0.2692
Val Loss: 2.0781, Val Acc: 0.2158
Precision: 0.7128, Recall: 0.2158, F1-score: 0.1114
--------------------------------------------------
Epoch [2/1500]
Train Loss: 1.9340, Train Acc: 0.3902
Val Loss: 2.0748, Val Acc: 0.2774
Precision: 0.5791, Recall: 0.2774, F1-score: 0.1890
--------------------------------------------------
Epoch [3/1500]
Train Loss: 1.8635, Train Acc: 0.4408
Val Loss: 2.0659, Val Acc: 0.3493
Precision: 0.4859, Recall: 0.3493, F1-score: 0.2967
--------------------------------------------------
Epoch [4/1500]
Train Loss: 1.8065, Train Acc: 0.4983
Val Loss: 2.0479, Val Acc: 0.4212
Precision: 0.4993, Recall: 0.4212, F1-score: 0.3933
--------------------------------------------------
Epoch [5/1500]
Train Loss: 1.7476, Train Acc: 0.5584
Val Loss: 2.0123, Val Acc: 0.4521
Precision: 0.5314, Recall: 0.4521, F1-score: 0.4264
--------------------------------------------------
Epoch [6/1500]
Train Loss: 1.7005, Train Acc: 0.59

In [ ]:
torch.save(model.state_dict(), "C:/Nini/Capstone/Models/LSTM_model_mel_mfcc_chroma.pth")